In [2]:
# Importing libraries
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

import math

from datetime import datetime, timedelta

from yahoofinancials import YahooFinancials
from joblib import Memory

In [4]:
# Create a list of stocks from the directory. These will be used to while creating a dataframe
list_of_stocks = os.listdir("Data/Stocks")

In [172]:
# Create Stock Dataframe. Take the first n stocks from the list and create a dataframe from them
def create_stock_dataframe(n):
    stocks = pd.DataFrame({'Ticker' : []})
    
    for i in range(0, n):
        # Read through the documents, adding in 
        data = pd.read_csv('Data/Stocks/' + list_of_stocks[i])
        data['Date'] = pd.to_datetime(data['Date'])
        data = data.set_index('Date')
        ticker =  list_of_stocks[i].split('.')[0]
        data = data['2016':'2017'].drop(['Open', 'High', 'Low', 'Volume', 'OpenInt'], axis=1).T
        data.insert(0, 'Ticker', ticker)
        data.insert(1, 'Pattern', "")
        stocks = pd.concat([stocks, data])
    return stocks

In [173]:
stock_data = create_stock_dataframe(15)
stock_data.head()

,Ticker,Pattern,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-11 00:00:00,2016-01-12 00:00:00,2016-01-13 00:00:00,...,2017-10-30 00:00:00,2017-10-31 00:00:00,2017-11-01 00:00:00,2017-11-02 00:00:00,2017-11-03 00:00:00,2017-11-06 00:00:00,2017-11-07 00:00:00,2017-11-08 00:00:00,2017-11-09 00:00:00,2017-11-10 00:00:00
Close,a,,39.979,39.842,40.018,38.317,37.916,37.278,37.523,36.216,...,67.49,68.03,68.02,68.13,68.37,68.22,68.25,68.11,67.47,66.81
Close,aa,,21.365,20.398,18.946,18.197,17.757,17.603,16.018,15.688,...,47.42,47.78,47.70,47.46,47.12,47.29,46.97,45.89,43.33,43.01
Close,aaap,,30.670,30.880,29.960,29.650,28.000,26.790,24.710,26.090,...,80.50,81.00,81.03,81.00,80.99,80.96,81.00,80.93,81.00,80.98
Close,aaba,,31.400,32.200,32.160,30.160,30.630,30.170,30.690,29.440,...,68.86,70.12,70.59,70.51,70.23,71.71,72.22,71.16,70.19,70.56
Close,aac,,18.520,19.460,19.010,18.200,18.110,17.460,16.900,16.240,...,8.20,7.92,7.55,10.02,10.08,11.14,11.07,11.11,10.88,11.28


In [149]:
def create_charts(data):
    data.iloc[1].plot() 
    plt.title('Closing price of' + '' + 'over time (Monthly frequency)')

In [175]:
stock_data['Ticker'][0]

'a'

In [183]:
%%capture
fig = plt.figure(figsize=(40, 20))
stock_data.iloc[0][2:-1].interpolate().plot()

plt.savefig('Images/' + stock_data['Ticker'][0] + '.png')